## Train My Model

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import datasets

# Load the diabetes dataset
diabetes = datasets.load_diabetes()

# ONLY USING 1 FEATURE FOR THIS EXAMPLE!
# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

# Create linear regression model
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(diabetes_X_train, diabetes_y_train)

## Save/Pickle Underlying Model as `model.pkl`

In [ ]:
import dill as pickle

model_pkl_path = 'model.pkl'

with open(model_pkl_path, 'wb') as fh:
    pickle.dump(model, fh)

In [ ]:
import subprocess

output = subprocess.check_output('ls -l model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)
print(output.decode('utf-8'))

## Initialize Model by Unpickling Underlying Model

In [ ]:
def initialize_model(model_pkl_path):
    import dill as pickle

    with open(model_pkl_path, 'rb') as fh:
        model = pickle.load(fh)

    return model            

In [ ]:
model = initialize_model('./model.pkl')
print(model)

## Transform `json` -> `nparray`

In [ ]:
def transform_request(request):
    import ujson
    import numpy as np
    request_str = request.decode('utf-8')
    request_str = request_str.strip().replace('\n', ',')
    # surround the json with '[' ']' to prepare for conversion
    request_str = '[%s]' % request_str
    request_json = ujson.loads(request_str)
    request_transformed = ([json_line['feature0'] for json_line in request_json])
    return np.array(request_transformed)

## Transform `nparray` -> `json`

In [ ]:
def transform_response(response):
    import ujson
    return ujson.dumps(response.tolist())

In [ ]:
def predict(model, 
            request):

    return model.predict(request)

In [ ]:
class PioModel(object):
    def __init__(self,
                 transform_request,
                 transform_response,
                 model,
                 predict):
        self.transform_request = transform_request
        self.transform_response = transform_response
        self.model = model
        self.predict = predict

In [ ]:
pio_model = PioModel(transform_request,
                     transform_response,
                     model,
                     predict)

## Save/Pickle `PioModel` as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
%%bash

ls -l pio_model.pkl